In [9]:
import pandas as pd ; import requests ; import io ; import re
import nltk ; from nltk.corpus import stopwords ; import string
import itertools ; import os ; from tqdm import tqdm ; import json
from wordcloud import WordCloud ; import matplotlib.pyplot as plt
import datetime ; import numpy as np ; from collections import Counter
import shifterator as sh ; import networkx as nx ; import netwulf as nw
from netwulf import visualize ; from networkx import Graph ; import random
from community import community_louvain ; import matplotlib as mpl
seed = 42
np.random.seed(seed)
random.seed(seed)


#url1 = "https://raw.githubusercontent.com/JaQtae/SocInfo2022/FinalProject/politics_subs_smol.csv"
#url2 = "https://raw.githubusercontent.com/JaQtae/SocInfo2022/FinalProject/politics_comments.csv"

#download_sub = requests.get(url1).content
#download_com = requests.get(url2).content

#sub_data = pd.read_csv(io.StringIO(download_sub.decode('utf-8')))
#com_data = pd.read_csv(io.StringIO(download_com.decode('utf-8')))


In [4]:
# Since json was corrupted, we redo week 3
sub_data = pd.read_csv('politics_subs_smol.csv', 
                      parse_dates = ['Unnamed: 0'], sep=',').set_index('Unnamed: 0')

sub_data.index = sub_data.index.rename('dates')
com_data = pd.read_csv('politics_comments.csv', 
                       parse_dates = ['Unnamed: 0'], sep=',').set_index('Unnamed: 0')
com_data.index = com_data.index.rename('dates')


com_authors = dict(zip(com_data['id'], com_data['author']))
parent = dict(zip(com_data['id'], com_data['parent_id']))
sub_authors = dict(zip(sub_data['id'], sub_data['author']))

def authID(com_id):
    if parent[com_id][:3] == "t1_":
        auth = com_authors.get(parent[com_id][3:], None)
    else:
        auth = sub_authors.get(parent[com_id][3:], None)
    return auth

com_data['parent_author'] = com_data['id'].apply(authID)
# Index below is sliced so it can actually show us a graph, otherwise computer died.
cd_ = com_data[(com_data.index < '2021-12-01')]
cd_ = cd_.drop(cd_[cd_['author'] == '[deleted]'].index) # remove deleteds
cd_ = cd_.drop(cd_[cd_['parent_author'] == '[deleted]'].index) # Remove deleteds
cd_ = cd_.drop(cd_[cd_['parent_author'].isna()].index) # Remove NaN's

In [19]:
sub_data

,title,id,score,author,num_comments
dates,,,,,
2020-11-02 22:54:58,Trump ramps up Fauci attacks on eve of electio...,jmybs3,1,geoxol,33
2020-11-02 22:48:58,Trump Loves To Declare Victory Even if He Didn...,jmy7vu,1,Facerealityalready,16
2020-11-02 22:46:57,Trump creates 1776 Commission to promote 'patr...,jmy6j9,1,bluestblue,53
2020-11-02 22:45:25,"25+ Reasons to VOTE BIDEN (instead of Trump, n...",jmy5ir,1,YouNeedToCalmDown24,2
2020-11-02 22:42:26,Judge blocks Trump campaign challenge to Nevad...,jmy3hn,1,TrumpSharted,10
...,...,...,...,...,...
2020-09-30 22:11:08,Judge rules Trump admin failed to justify Muel...,j2vwwt,1,Plymouth03,587
2020-09-30 22:10:32,Track your CA ballot just in case Supreme Lead...,j2vwhq,1,SPANXmomIneededTHAT,1
2020-09-30 22:09:38,"Biden Tours Ohio and Pennsylvania, Aiming to W...",j2vvvq,1,DonnieMostDefinitely,5


In [20]:
com_data

,id,link_id,score,author,parent_id,parent_author,time
dates,,,,,,,
2020-11-04 21:22:44,gb5q6y3,t3_jmybs3,1,[deleted],t3_jmybs3,geoxol,1604524964
2020-11-03 01:38:03,gaykxxw,t3_jmybs3,1,saint-cecelia,t1_gayg0dt,NorweAmeriLove,1604367483
2020-11-03 00:52:13,gayg0dt,t3_jmybs3,1,NorweAmeriLove,t1_gay44pe,saint-cecelia,1604364733
2020-11-03 00:48:23,gayflfy,t3_jmybs3,1,ChickenNPisza,t1_gaye3oj,droplivefred,1604364503
2020-11-03 00:34:43,gaye3oj,t3_jmybs3,1,droplivefred,t1_gay339x,x_______name,1604363683
...,...,...,...,...,...,...,...
2020-09-30 22:54:31,g789orr,t3_j2vsm6,1,[deleted],t1_g78508x,beaucephus,1601506471
2020-09-30 22:13:21,g785des,t3_j2vsm6,1,jchrist1225,t1_g78508x,beaucephus,1601504001
2020-09-30 22:10:43,g78508x,t3_j2vsm6,1,beaucephus,t3_j2vsm6,monkeygsun06472,1601503843


In [12]:
# We make a directed graph. 
edges = cd_.groupby(['author', 'parent_author']).count()
edges = edges.drop(['id', 'link_id', 'parent_id'], axis=1)
G = nx.DiGraph()

# Add all edges and remake to reciprocal.
#G.add_weighted_edges_from([ (a, b, edges['score'].loc[(a,b)]) for a, b in edges.index ])
G.add_edges_from((a,b) for a,b in edges.index)

#G_U = G.to_undirected(reciprocal=True)
#for node, ngbr, w in G_U.edges.data('weight'):
    # Undirected edge weights equal to sum of weights in two corresponding directed edges
#    G_U.edges[node, ngbr]['weight'] = (G.edges[node, ngbr]['weight'] + G.edges[ngbr, node]['weight'])

# Finally, remove isolated nodes and self-loops in graph.
G.remove_nodes_from(list(nx.isolates(G_U))) 
#G.remove_edges_from(nx.selfloop_edges(G_U))

print('Nodes: {}'.format(G.number_of_nodes()))
print('Edges: {}'.format(G.number_of_edges()))
#print('Fraction of nodes compared to OG network: {:.3f}'.format(G_U.number_of_nodes()/G.number_of_nodes()))
#print('Fraction of edges compared to OG network: {:.3f}'.format(G_U.number_of_edges()/G.number_of_edges()))
print('Interpretation of results: \n Most of the nodes/users only commented on posts without getting replies. Cases with 2 users were reduced to 1 edge, and commenters without replies have been deleted.')

Nodes: 240139
Edges: 1173793
Interpretation of results: 
 Most of the nodes/users only commented on posts without getting replies. Cases with 2 users were reduced to 1 edge, and commenters without replies have been deleted.


In [18]:
L = len(G.edges)
N = len(G.nodes)
k = L/N
k
#p = 2*L/(N*(N-1))
#print(f'p={p * 100:.2f}%')
#k = 2*L/N
#print('k: {:.3f}'.format(k))

4.887973215512682